In [ ]:
import pandas as pd
import numpy as np
import scipy

In [ ]:
from sklearn.linear_model import Ridge,LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer

In [ ]:
#loading data
train = pd.read_csv('../input/train.tsv', sep = '\t')
test = pd.read_csv('../input/test.tsv', sep = '\t')


In [ ]:
NUM_BRANDS = 2500
NAME_MIN_DF = 10
MAX_FEAT_DESCP = 50000

In [ ]:
#train.head(3)
train.brand_name.value_counts()

In [ ]:
print(test.head(3))
print (test.shape)

In [ ]:
#concatenating test and train into a single dataset
dt = pd.concat([train,test],0)


In [ ]:
dt.head(3)


In [ ]:
dt.brand_name.value_counts()

In [ ]:
#this gives the number of rows in the training set
nrow_train = train.shape[0]
#print (nrow_train)


In [ ]:
#converting the target variable "price" into it's log values for better fit
target = np.log1p(train.price)
target

In [ ]:
#filling names and missing values in category_name
dt.category_name = dt.category_name.fillna('Other').astype('category')
dt.brand_name = dt.brand_name.fillna('unknown')

In [ ]:
dt.isnull().sum()

In [ ]:
pop_brands = dt.brand_name.value_counts().index[:NUM_BRANDS]

In [ ]:
pop_brands

In [ ]:
dt.loc[~dt.brand_name.isin(pop_brands), 'brand_name'] = 'Other'

In [ ]:
dt.head(3)

In [ ]:
dt.item_description = dt.item_description.fillna('None')
dt.item_condition_id = dt.item_condition_id.astype('category')
dt.brand_name = dt.brand_name.astype('category')


In [ ]:
#encoding name
count = CountVectorizer(min_df = NAME_MIN_DF)
x_name = count.fit_transform(dt.name)

In [ ]:
x_name

In [ ]:
from scipy.sparse import find

In [ ]:
find(x_name)

In [ ]:
#encoding category variables
#spliting data info in category_name via split('/)
unique_categories = pd.Series('/'.join(dt.category_name.unique().astype('str')).split('/')).unique()

In [ ]:
unique_categories.take(10)

In [ ]:
count_category = CountVectorizer()

In [ ]:
x_category = count_category.fit_transform(dt.category_name)

In [ ]:
#item_description
count_desc = TfidfVectorizer(max_features = MAX_FEAT_DESCP,
                            ngram_range = (1,3),
                            stop_words = 'english')


In [ ]:
x_descp = count_desc.fit_transform(dt.item_description)

In [ ]:
#brand_name encoder
vect_brand = LabelBinarizer(sparse_output = True)
x_brand = vect_brand.fit_transform(dt.brand_name)

In [ ]:
#dummy encoders
x_dummies = scipy.sparse.csr_matrix(pd.get_dummies(dt[['item_condition_id','shipping']],sparse = True).values)

In [ ]:
X = scipy.sparse.hstack((x_dummies,
                        x_descp,
                        x_brand,
                        x_category,
                        x_name)).tocsr()

In [ ]:
print (x_dummies.shape, x_category.shape, x_name.shape,x_descp.shape,x_brand.shape)

In [ ]:
x_train = X[:nrow_train]

# RIDGE

In [ ]:
clf = Ridge(solver = 'lsqr',fit_intercept = False)

In [ ]:
y_train = target

In [ ]:
#fitting classifier
clf.fit(x_train,y_train)

In [ ]:
x_test = X[nrow_train:]

In [ ]:
preds = clf.predict(x_test)

In [ ]:
test["price"] = np.expm1(preds)
test[['test_id','price']].to_csv('ridge_clf.csv',index = False)